# Easy Stable Diffusion
＊ ⚡**需確保連線包含GPU**<br>
＊ 此為分享個人研究測試，不負任何責任，隨性維護<br>

製作 By [鱷魚俠](https://www.youtube.com/channel/UCzJPta8YRVErK49LvyxOt3Q?sub_confirmation=1)

# 一鍵啟動

In [ ]:
#@title 啟動Stable Diffusion
#@markdown # <-- 點這裡啟動

#@markdown ---
#@markdown 預設尺寸(寬x高)<br>
size = "512x768" #@param ["512x512", "512x768", "768x512", "512x910", "910x512"] {allow-input: true}

#@markdown ---
#@markdown 可自行更換 Model<br>
model_name = "chilloutmix_NiPrunedFp32Fix.safetensors" #@param {'type': 'string'}
model_url = "https://civitai.com/api/download/models/11745" #@param {'type': 'string'}

loramodel_name = "Moxin_10.safetensors" #@param {'type': 'string'}
loramodel_url = "https://civitai.com/api/download/models/14856" #@param {'type': 'string'}

#@markdown ---
#@markdown Style 1<br>
style_name_1 = "Sample" #@param {'type': 'string'}
prompt_1 = "RAW photo, delicate, best quality, (intricate details:1.3), (hyper detail), colorful, 1girl, cute, 8K UHD, film grain, (studio lighting:1.2), (Fujifilm XT3), (photorealistic:1.3), (detailed skin:1.2), (full body), see-through, dating, (nose blush), (smile:1.23),(closed mouth), middle breasts, beautiful detailed eyes, daylight, (outdoor), (short sleeve cardigan:1.2), (short hair:1.1), (park)," #@param {'type': 'string'}
negative_prompt_1 = "(nsfw:1.2), paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), manboobs, lowres, ((monochrome)), (grayscale:1.2), backlight, (ugly:1.331), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (logo:1.2), (watermark:1.2),(letters:1.2),teeth,mole,multiple breasts, (mutated hands and fingers:1.5 ), (long body :1.3), (mutation, poorly drawn :1.2) ,blurred, (fused fingers:1.5), (too many fingers:1.5), (unclear eyes:1.3), bad hands, missing fingers,more than 2 nipples, different nipples,more than 1 left hand,more than 1 right hand,more than 2 thighs,more than 2 legs,worst quality,low quality,normal quality, ng_deepnegative_v1_75t" #@param {'type': 'string'}

#@markdown Style 2<br>
style_name_2 = "a cool dog" #@param {'type': 'string'}
prompt_2 = "1 dog, fantasy art drawn, cool, delicate, movie concept art, cinematic composition, best quality, (intricate details:1.3), (hyper detail), colorful, cute, (studio lighting:1.2), (detailed skin:1.2), see-through,(full body), castle" #@param {'type': 'string'}
negative_prompt_2 = "(nsfw:1.2), sketches, (worst quality:2), (low quality:2), (normal quality:2), manboobs, lowres, ((monochrome)), (grayscale:1.2), backlight, (ugly:1.331), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (logo:1.2), (watermark:1.2),(letters:1.2),teeth,mole,multiple breasts, (mutated hands and fingers:1.5 ), (long body :1.3), (mutation, poorly drawn :1.2) ,blurred, (fused fingers:1.5), (too many fingers:1.5), (unclear eyes:1.3), bad hands, missing fingers,more than 2 nipples, different nipples,more than 1 left hand,more than 1 right hand,more than 2 thighs,more than 2 legs,worst quality,low quality,normal quality, ng_deepnegative_v1_75t" #@param {'type': 'string'}

#@markdown ---

import os.path
import json
import os

type_ckpt = 0
type_vae = 1
type_embedding = 2
type_lora = 3

root_path = "/content"
sd_path = root_path+"/stable-diffusion-webui"

def dlm(name, url, mtype):
  path = sd_path
  if mtype == type_ckpt:
    path = path+"/models/Stable-diffusion/"
  if mtype == type_vae:
    path = path+"/models/VAE/"
  elif mtype == type_embedding:
    path = path+"/embeddings/"
  elif mtype == type_lora:
    path = path+"/models/Lora/"
  !curl -Lo {root_path}/{name} {url}
  !mv {root_path}/{name} {path}

def dlt(url, path):
  !curl -Lo {path} {url}

def set_style():
  fp = open(sd_path+"/styles.csv", "w")
  fp.write("name,prompt,negative_prompt\n")
  
  if style_name_1 != "" and prompt_1 != "" and negative_prompt_1 != "":
    fp.write(style_name_1+","+"\""+prompt_1+"\",\""+negative_prompt_1+"\"\n")

  if style_name_2 != "" and prompt_2 != "" and negative_prompt_2 != "":
    fp.write(style_name_2+","+"\""+prompt_2+"\",\""+negative_prompt_2+"\"") 
  fp.close()
  return

def set_config():
  if os.path.isfile(sd_path+"/webui.bat") == False:
    with open(sd_path+"/ui-config.json", "w") as outfile:
        outfile.write("")
  try :
    with open(sd_path+"/ui-config.json") as f:
      data = json.load(f)
  except:
    data = {}

  data['txt2img/Restore faces/value'] = True
  data['txt2img/CFG Scale/value'] = 8.5

  setSize = size.split('x')
  data['txt2img/Width/value'] = int(setSize[0])
  data['txt2img/Height/value'] = int(setSize[1])

  s = json.dumps(data)

  with open(sd_path+"/ui-config.json", "w") as outfile:
      outfile.write(s)
  return

def install_sd():
  %cd /content
  !pip install --upgrade fastapi==0.90.1
  !git clone https://github.com/crocranger/stable-diffusion-webui.git
  !git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser 
  
  !mkdir {sd_path}/models/Lora

  dlm("vae-ft-mse-840000-ema-pruned.ckpt", "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt", type_vae)
  dlm("ng_deepnegative_v1_75t.pt", "https://civitai.com/api/download/models/5637", type_embedding)

  if model_name != "" and model_url != "":
    dlm(model_name, model_url, type_ckpt)

  if loramodel_name != "" and loramodel_url != "":
    dlm(loramodel_name, loramodel_url, type_lora)

  set_style()
  set_config()

  %cd /content/stable-diffusion-webui
  !COMMANDLINE_ARGS="--share --disable-safe-unpickle --skip-torch-cuda-test --no-half-vae --xformers --reinstall-xformers --enable-insecure-extension-access" REQS_FILE="requirements.txt" python launch.py
  return

def run_sd():
  set_style()
  %cd /content/stable-diffusion-webui 
  !COMMANDLINE_ARGS="--share --disable-safe-unpickle --skip-torch-cuda-test --no-half-vae --xformers --enable-insecure-extension-access" REQS_FILE="requirements.txt" python launch.py
  return

if os.path.isfile(sd_path+"/webui.bat"):
  run_sd()
else:
  install_sd()

In [ ]:
#@title 儲存產生的圖片
#@markdown #<-- 點這裡壓縮所有圖檔
#@markdown ＊ 壓縮後的檔案存於根目錄
save_folder = "txt2img-images" #@param ["txt2img-images","img2img-grids"] {allow-input: true}

import time
from google.colab import files

localtime = time.localtime(time.mktime(time.localtime()) + 28800)

%cd /content/stable-diffusion-webui/outputs
#%cd /content/test/
nowTime = time.strftime("_%Y%m%d_%H%M", localtime)
zipFileName = "/content/output"+nowTime+".zip"
!zip -r {zipFileName} {save_folder}


from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
def create_folder(folder_name):
    # Check if folder exists
    file_list = drive.ListFile({'q': "title='{}' and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_name)}).GetList()
    if len(file_list) > 0:
        # Folder exists
        print('Debug: Folder exists')
        folder_id = file_list[0]['id']
    else:
        print('Debug: Creating folder')
        file = drive.CreateFile({'title': folder_name, 'mimeType': 'application/vnd.google-apps.folder'})
        file.Upload()
        folder_id = file.attr['metadata']['id']
    # return folder id
    return folder_id
# Upload file to Google Drive
def upload_file(file_name, folder_id, save_as):
    # Check if file exists
    file_list = drive.ListFile({'q': "title='{}' and trashed=false".format(save_as)}).GetList()
    if len(file_list) > 0:
        print('Debug: File already exists')
        # Change file name to avoid overwriting
        save_as = save_as + ' (1)'
    file = drive.CreateFile({'title': save_as, 'parents': [{'id': folder_id}]})
    file.SetContentFile(file_name)
    # Upload and set permission to public
    file.Upload()
    file.InsertPermission({'type': 'anyone', 'value': 'anyone', 'role': 'reader'})
    # return file id
    return file.attr['metadata']['id']

#@markdown * 上傳至GoogleDrive
enable_google_drive = False #@param {type:"boolean"}
#@markdown * 匯出目錄名稱
folder_name = "AI picture" #@param {type: "string"}
#@markdown * 匯出檔名
file_name = "output" #@param {type: "string"}
#@markdown * 檔名後面加時間戳
add_time = True #@param {type:"boolean"}

if enable_google_drive:
  gauth = GoogleAuth()
  auth.authenticate_user()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  outputFileName = file_name
  if add_time:
    nowTime = time.strftime("_%Y%m%d_%H%M", localtime)
    outputFileName = outputFileName+nowTime
  file_id = upload_file(zipFileName, create_folder(folder_name), outputFileName+".zip")
  print("Your sharing link: https://drive.google.com/file/d/" + file_id + "/view?usp=sharing")

dir = "/content/stable-diffusion-webui/outputs/"+save_folder
toDayTime = time.strftime("/%Y-%m-%d", localtime)
dir = dir+toDayTime
for f in os.listdir(dir):
  if f.endswith(".png"):
    print(f)
    os.remove(os.path.join(dir, f))

# 擴充功能

In [ ]:
#@title 安裝模型
#@markdown ###1. 選擇模型類型
#@markdown ###2. 輸入模型名稱含副檔名
#@markdown ###3. 輸入下載URL

import os.path

type_ckpt = 0
type_vae = 1
type_embedding = 2
type_lora = 3

root_path = "/content"
sd_path = root_path+"/stable-diffusion-webui"

model_type = "Check Point" #@param ["Check Point", "Textual Inversion", "Lora"] 
model_name = "Deliberate_v2.safetensors" #@param {'type': 'string'}
model_url = "https://huggingface.co/XpucT/Deliberate/resolve/main/Deliberate_v2.safetensors" #@param {'type': 'string'}

def dlm(name, url, mtype):
  path = sd_path
  if mtype == type_ckpt:
    path = path+"/models/Stable-diffusion/"
  if mtype == type_vae:
    path = path+"/models/VAE/"
  elif mtype == type_embedding:
    path = path+"/embeddings/"
  elif mtype == type_lora:
    path = path+"/models/Lora/"
  !curl -Lo {root_path}/{name} {url}
  !mv {root_path}/{name} {path}

if model_type != "" and model_name != "" and model_url != "":
  if model_type == "Check Point":
    dlm(model_name, model_url, type_ckpt)
  if model_type == "Textual Inversion":
    dlm(model_name, model_url, type_embedding)
  if model_type == "Lora":
    dlm(model_name, model_url, type_lora)

In [ ]:
#@title 安裝模型從目錄

import os.path

type_ckpt = 0
type_vae = 1
type_embedding = 2
type_lora = 3

root_path = "/content"
sd_path = root_path+"/stable-diffusion-webui"
ckpt_path = sd_path+"/models/Stable-diffusion/"
vae_path = sd_path+"/models/VAE/"
embeddings_path = sd_path+"/embeddings/"
lora_path = sd_path+"/models/Lora/"

ckpt_folder = root_path+"/ckpt"
vae_folder = root_path+"/vae"
embeddings_folder = root_path+"/embeddings"
lora_folder = root_path+"/lora"

def loadCKPT():
  if os.path.isdir(ckpt_folder):
    files = os.listdir(ckpt_folder)
    for f in files:
      !mv {ckpt_folder}/{f} {ckpt_path}
  return

def loadVAE():
  if os.path.isdir(vae_folder):
    files = os.listdir(vae_folder)
    for f in files:
      !mv {vae_folder}/{f} {vae_path}
  return

def loadEmbedings():
  if os.path.isdir(embeddings_folder):
    files = os.listdir(embeddings_folder)
    for f in files:
      !mv {embeddings_folder}/{f} {embeddings_path}
  return 

def loadLora():
  if os.path.isdir(lora_folder):
    files = os.listdir(lora_folder)
    for f in files:
      !mv {lora_folder}/{f} {lora_path}
  return

loadCKPT()
loadVAE()
loadEmbedings()
loadLora()

# 系統檢查

In [ ]:
#@title 系統資訊
#@markdown #顯示顯卡資訊
import os

!nvidia-smi

In [ ]:
#@title 更新Python
#@markdown #更新至3.10<br>
#@markdown 會花約２分鐘，但會減少一些錯誤


!python -V
!wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
!bash ./py310.sh -b -f -p /usr/local
!python -m ipykernel install --name "py310" --user
!python -V

In [ ]:
#@title 除錯用
#@markdown #出問題才需要執行
import os

!nvidia-smi
!pip install pytorch_lightning
import pytorch_lightning as pl
print(pl.__version__)

# 參考資源
＊ 如作者移除下載路徑，有可能造成安裝失敗<br>

Reference resources：
1. [Stable diffusion webui](https://github.com/AUTOMATIC1111/stable-diffusion-webui)
2. [Chilloutmix-ni](https://civitai.com/models/6424/chilloutmix)
3. [NG Deepnegative](https://civitai.com/models/4629/deep-negative-v1x)
4. [VAE](https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt)
5. [墨心 MoXin](https://civitai.com/models/12597/moxin)